# 📌 Do you calculate Gini conservatively?

**Topic:** Ties, CAP vs ROC, and a robust correction

**Author:** Alexey Mengden (FRM, CQF)  
**Repo:** _<GitLab project URL or relative path>_  
**Last updated:** 2026-01-14

> **Goal:** Provide clean, reproducible code + visuals that support the carousel.

---

## Table of contents
1. [Context](#context)  
2. [Setup](#setup)  
3. [Problem statement](#problem-statement)  
4. [Theory cheat-sheet](#theory-cheat-sheet)  
5. [Implementation](#implementation)  
6. [Experiments & results](#experiments--results)  
7. [Validation & sanity checks](#validation--sanity-checks)  
8. [Key takeaways](#key-takeaways)  
9. [References](#references)


In [1]:
# --- Environment & reproducibility ---
import numpy as np
import pandas as pd

from validation_series_quant.vs01_gini_ties import metrics

rng = np.random.default_rng(42)
print("✅ Setup complete")


✅ Setup complete


## Problem statement

The Gini coefficient is widely used to measure the discriminatory power of Probability of Default (PD) models. <br>
It can be defined via the Cumulative Accuracy Profile (CAP) as the ratio between the area under the model curve and the diagonal, relative to the area between the perfect model and the diagonal. <br> This metric — also called the Accuracy Ratio — can be computed numerically using the trapezoidal rule:
\begin{equation}
\textrm{Gini} =  \frac{\sum_{i}{(y_i+y_{i+1})(x_{i+1}-x_{i})}-1}{1-\textrm{DR}}
\end{equation}
where: <br>
$N$ — the number of observations; <br>
$i$ — the serial number of the observation in the cumulative share; <br>
$X_i$ — the cumulative share of all observations ($X_0 = 0$); <br>
$Y_i$ — the cumulative share of observations in default ($Y_0 = 0$);  <br>
$\mathrm{DR}$ — the default rate. <br>
The cumulative shares are calculated in ascending (descending) order of the predicted value: if the higher the predicted values in the probability of default assessment model, the lower (higher) the probability of default. <br>
Alternatively, Gini can be derived from the Area Under the Receiver Operating Characteristic Curve (AUC-ROC):
\begin{equation}
\textrm{Gini} = 2\times\textrm{AUC-ROC} - 1
\end{equation}
However, when defaults and non-defaults share the same score (ties), standard implementations can be optimistic and depend on sampling order. If there are several options for ranking borrowers (credit claims) in the sample for calculating the Gini coefficient, it is natural that the metric should be calculated using **a conservative approach** (the approach that results in the lowest value of the Gini coefficient should be used). This is appropriate for the IRB approach.


---


## Solution

To remove sampling-order variability and obtain the most conservative Gini, apply the tie-breaking rule (defaults last). For AUC-based computation, subtract $\frac{T}{N_{-}N_{+}}$ from the standard Gini. 


|              | **from CAP** | **from ROC** | 
| --------     | --------     | --------     |
| **Conservative correction** | sort that defaults occur later in tie group | $-\frac{T}{N_{-}N_{+}}$ |



---


## Experiments & results



### [*1] Generate Data with Ties

In [2]:
N = 5000
p_bad = 0.2  # share of class 1 (bad/positive event)

y = (rng.random(N) < p_bad).astype(int)

# "score": signal + noise
raw = 0.8 * y + rng.normal(0.0, 1.0, size=N)

# create ties
s = np.round(raw, 1)

# sanity: share of ties
n_unique = np.unique(s).size
print(f"N={N}, unique scores={n_unique}, ties ratio={(1 - n_unique/N):.3f}")

df = pd.DataFrame({"y": y, "s": s})
df.head()


N=5000, unique scores=73, ties ratio=0.985


,y,s
0,0,-1.1
1,0,-0.5
2,0,0.3
3,0,-0.6
4,1,0.5


In [3]:
target_col = "y"
predicted_col = "s"


### [*2] AUC-based Gini: standard vs conservative correction

In [4]:
g_std = metrics.gini_auc(df[target_col], df[predicted_col], conservative_ties=False)
g_cons = metrics.gini_auc(df[target_col], df[predicted_col], conservative_ties=True)

print(f"gini_auc standard      = {g_std:.6f}")
print(f"gini_auc conservative  = {g_cons:.6f}")


gini_auc standard      = 0.423453
gini_auc conservative  = 0.399164


### [*3] CAP-based Gini: tie-break sensitivity (stable vs conservative vs optimistic)

In [5]:
g_cap_stable = metrics.gini_cap(df[target_col], df[predicted_col], tie_break="stable")
g_cap_cons = metrics.gini_cap(df[target_col], df[predicted_col], tie_break="conservative")
g_cap_opt = metrics.gini_cap(df[target_col], df[predicted_col], tie_break="optimistic")

print(f"gini_cap stable        = {g_cap_stable:.6f}")
print(f"gini_cap conservative  = {g_cap_cons:.6f}")
print(f"gini_cap optimistic    = {g_cap_opt:.6f}")


gini_cap stable        = 0.423336
gini_cap conservative  = 0.399164
gini_cap optimistic    = 0.447741


### [*4] Permutation test: CAP(stable) depends on row order, correction does not

In [6]:
def cap_stable_under_permutation(df_in, n_perm=50):
    vals = []
    for _ in range(n_perm):
        idx = rng.permutation(len(df_in))
        d = df_in.iloc[idx]
        vals.append(metrics.gini_cap(d["y"].values, d["s"].values, tie_break="stable"))
    return np.array(vals)

vals = cap_stable_under_permutation(df, n_perm=50)
print(
    f"CAP stable over perms: min={vals.min():.6f},
    mean={vals.mean():.6f},
    max={vals.max():.6f},
    spread={vals.max()-vals.min():.6f}"
)


CAP stable over perms: min=0.422320, mean=0.423544, max=0.424865, spread=0.002545


## References

- _<https://www.linkedin.com/posts/alexey-mengden_gini-conservative-handling-of-ties-activity-7381397727777255424-dFuH?utm_source=share&utm_medium=member_desktop&rcm=ACoAAFbp6qYBEB_ykfytA7IUHlecuioC7Dl2D0w>_



